Goals of this project:
1. How much impact does being late or too spaced out at the first stop have downstream?
2. What is the impact of the layover at the start of the trip (the difference between the first top arrival and departure time)? Does more layover lead to more stable headways (lower values for % headway deviation)?
3. How closely does lateness (ADHERENCE) correlate to headway?
4. What is the relationship between distance or time travelled since the start of a given trip and the headway deviation? Does headway become less statble the further along the route the bus has travelled?
5. How much of a factor does the driver have on headway and on-time performance? The driver is indicated by the OPERATOR variable.
6. How does direction of travel, route, or location affect the headway and on-time performance?
7. How does time of day or day of week affect headway and on-time performance? Can you detect an impact of school schedule on headway deviation (for certain routes and at certain times of day)?
8. Does weather have any effect on headway or on-time performance? To help answer this question, the file bna_2022.csv contains historical weather data recorded at Nashville International Airport.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
from datetime import date
from datetime import datetime
plt.style.use('ggplot')

%matplotlib inline

### Access, Display DataFrames

In [ ]:
# Access 'Headway Data.csv'
headway = pd.read_csv('../data/Headway Data.csv')
headway.head()

In [ ]:
# Access 'bna_2022.csv'
bna_2022 = pd.read_csv('../data/bna_2022.csv')
bna_2022.head()

In [ ]:
# Access 'bna_weather.csv'
bna_weather = pd.read_csv('../data/bna_weather.csv')
bna_weather.head()

### Exploratory Data Analysis

#### Categorical Variables (Bus Routes)

Look for data types using `.info()`

In [ ]:
headway.info()

Count null values using `.isna().sum()`

In [ ]:
headway.isna().sum()

In [ ]:
# Identify the number of bus routes values within the dataset
headway['ROUTE_ABBR'].value_counts()

In [ ]:
# Display the number of entries within the dataset
headway['ROUTE_ABBR'].value_counts().plot(kind = 'bar',
                                         figsize = (8,4))

plt.xticks(rotation = 0,
           fontsize = 12)                   
plt.title('Entries by Bus Route',
         fontsize = 14,
         fontweight = 'bold');

#### Numeric Variables

In [ ]:
# Display the mean, median, max, min, standard deviation, and quartiles of adherence to the schedule
headway['ADHERENCE'].describe()

### Edit, Transform

In [ ]:
# Make a new 'headway' DataFrame with only the columns needed
headway_slim = headway.drop(headway.iloc[:, 22:30],axis = 1)
headway_slim.head(19)

In [ ]:
headway_slim.info()

In [ ]:
# # Drop NaN values from the 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 'HDWY_DEV' columns
# headway_slim.dropna(subset=['SCHEDULED_HDWY', 'ACTUAL_HDWY', 'HDWY_DEV'])
# # Show one to/from round to define impact
# headway_slim.head(10)

In [ ]:
# Verify that NaN values have been dropped from DataFrame
headway_slim.isna().sum()

#### Q1:
How much impact does being late or too spaced out at the first stop have downstream?

#### ADHERENCE quartiles

In [ ]:
# # Identify the quartiles for 'ADHERENCE' ('ACTUAL_DEPARTURE_TIME' - 'SCHEDULED_TIME')
# headway_slim['ADHERENCE'].describe()

In [ ]:
# For loop to identify  'ADHERENCE' ('ACTUAL_DEPARTURE_TIME' - 'SCHEDULED_TIME') quartile
# Consider an alternative to iterrows() to increase efficiency such as use for loop to create list, add list to DataFrame
headway_slim['ADHERENCE_QUARTILE'] = ''
for ind, row in headway_slim.iterrows():
    value = row['ADHERENCE']
    if pd.isna(value):
        headway_slim.loc[ind, 'ADHERENCE_QUARTILE'] = 'N/A'
    elif value < -3.4:
        headway_slim.loc[ind, 'ADHERENCE_QUARTILE'] = 'Slowest 25% Quartile'
    elif value < -1.383:
        headway_slim.loc[ind, 'ADHERENCE_QUARTILE'] = 'Median'
    elif value < -1.333333:
        headway_slim.loc[ind, 'ADHERENCE_QUARTILE'] = '50% - 75% Quartile'
    else:
        headway_slim.loc[ind, 'ADHERENCE_QUARTILE'] = 'Fastest 25% Quartile'
headway_slim.head()

In [ ]:
# Group 'ADHERENCE' averages by 'ADHERENCE_QUARTILE' and 'TRIP_EDGE'
headway_quartile = headway_slim.groupby(['ADHERENCE_QUARTILE', 'TRIP_EDGE']).agg({'ADHERENCE':['mean']})
headway_quartile

In [ ]:
# # Group 'ADHERENCE' averages by 'ADHERENCE_QUARTILE','TRIP_EDGE', 'ROUTE_ABBR'
# headway_quartile = headway_slim.groupby(['ADHERENCE_QUARTILE', 'TRIP_EDGE', 'ROUTE_ABBR']).agg({'ADHERENCE':['mean']})
# headway_quartile

#### HDWY_DEV quartiles

In [ ]:
# # Identify the quartiles for 'HDWY_DEV' ('ACTUAL_HDWY' - 'SCHEDULED_HDWY')
# headway_slim['HDWY_DEV'].describe()

In [ ]:
# For loop to identify  'HDWY_DEV' ('ACTUAL_HDWY' - 'SCHEDULED_HDWY') quartile
# Identify 'NaN' values and assign them 'N/A'
# Assign a quartile value to the rest of values that are not 'NaN'
headway_slim['HDWY_QUARTILE'] = ''
for ind, row in headway_slim.iterrows():
    value = row['HDWY_DEV']
    if pd.isnull(value):
        headway_slim.loc[ind, 'HDWY_QUARTILE'] = 'N/A'
    elif value < -1.88:
        headway_slim.loc[ind, 'HDWY_QUARTILE'] = 'Slowest 25% Quartile'
    elif value < 0:
        headway_slim.loc[ind, 'HDWY_QUARTILE'] ='Median'
    elif value < 1.9666666:
        headway_slim.loc[ind, 'HDWY_QUARTILE'] ='50% - 75% Quartile'
    else:
        headway_slim.loc[ind, 'HDWY_QUARTILE'] ='Fastest 25% Quartile'
headway_slim.head()

In [ ]:
# # Group 'HDWY_DEV' averages by 'HDWY_QUARTILE' and 'TRIP_EDGE'
# headway_quartile = headway_slim.groupby(['HDWY_QUARTILE', 'TRIP_EDGE']).agg({'HDWY_DEV':['mean']})
# headway_quartile

#### Q2:
> What is the impact of the layover at the start of the trip (the difference between the first stop arrival and departure time)?

> Does more layover lead to more stable headways (lower values for % headway deviation)?

In [77]:
# Create a dataframe with selected columns from original dataset
impact_df = headway[['ROUTE_ABBR', 'TRIP_ID', 'TRIP_EDGE', 'TIME_POINT_ABBR', 'ROUTE_STOP_SEQUENCE', 'ROUTE_DIRECTION_NAME', 'DWELL_IN_MINS', 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 'HDWY_DEV', 'DATE']]
impact_df.head()

,ROUTE_ABBR,TRIP_ID,TRIP_EDGE,TIME_POINT_ABBR,ROUTE_STOP_SEQUENCE,ROUTE_DIRECTION_NAME,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,DATE
0,7,297750,1,HBHS,4.0,TO DOWNTOWN,12.650000,14.0,15.983333,1.983333,2021-11-01
1,7,297750,0,21BK,3.0,TO DOWNTOWN,0.000000,14.0,17.333333,3.333333,2021-11-01
2,7,297750,2,MCC5_9,2.0,TO DOWNTOWN,22.416666,NaN,NaN,NaN,2021-11-01
3,50,297749,1,MLKS,7.0,TO DOWNTOWN,5.766666,NaN,NaN,NaN,2021-11-01
4,50,297749,2,MCC5_11,5.0,TO DOWNTOWN,0.000000,NaN,NaN,NaN,2021-11-01


In [78]:
# 1. Add 'DATE' column to the dataframe and change format to 'datetime'
# 2. Add 'WEEKDAY' column to dataframe by using a list comprehension
impact_df['DATE'] = pd.to_datetime(impact_df['DATE'])
impact_df['WEEKDAY'] = [calendar.day_name[x.weekday()] for x in impact_df['DATE']]
impact_df.head()

C:\Users\tyler\AppData\Local\Temp\ipykernel_15916\1973917356.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impact_df['DATE'] = pd.to_datetime(impact_df['DATE'])
C:\Users\tyler\AppData\Local\Temp\ipykernel_15916\1973917356.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impact_df['WEEKDAY'] = [calendar.day_name[x.weekday()] for x in impact_df['DATE']]


,ROUTE_ABBR,TRIP_ID,TRIP_EDGE,TIME_POINT_ABBR,ROUTE_STOP_SEQUENCE,ROUTE_DIRECTION_NAME,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,DATE,WEEKDAY
0,7,297750,1,HBHS,4.0,TO DOWNTOWN,12.650000,14.0,15.983333,1.983333,2021-11-01,Monday
1,7,297750,0,21BK,3.0,TO DOWNTOWN,0.000000,14.0,17.333333,3.333333,2021-11-01,Monday
2,7,297750,2,MCC5_9,2.0,TO DOWNTOWN,22.416666,NaN,NaN,NaN,2021-11-01,Monday
3,50,297749,1,MLKS,7.0,TO DOWNTOWN,5.766666,NaN,NaN,NaN,2021-11-01,Monday
4,50,297749,2,MCC5_11,5.0,TO DOWNTOWN,0.000000,NaN,NaN,NaN,2021-11-01,Monday


In [79]:
# Note that you can calculate headway deviation percentage as HDWY_DEV / SCHEDULED_HDWY.
# Add 'HDWY_DEV_PCT' column to impact_df
impact_df['HDWY_DEV_PCT'] = impact_df['HDWY_DEV'] / impact_df['SCHEDULED_HDWY']
impact_df.head()

C:\Users\tyler\AppData\Local\Temp\ipykernel_15916\564687872.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impact_df['HDWY_DEV_PCT'] = impact_df['HDWY_DEV'] / impact_df['SCHEDULED_HDWY']


,ROUTE_ABBR,TRIP_ID,TRIP_EDGE,TIME_POINT_ABBR,ROUTE_STOP_SEQUENCE,ROUTE_DIRECTION_NAME,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,DATE,WEEKDAY,HDWY_DEV_PCT
0,7,297750,1,HBHS,4.0,TO DOWNTOWN,12.650000,14.0,15.983333,1.983333,2021-11-01,Monday,0.141667
1,7,297750,0,21BK,3.0,TO DOWNTOWN,0.000000,14.0,17.333333,3.333333,2021-11-01,Monday,0.238095
2,7,297750,2,MCC5_9,2.0,TO DOWNTOWN,22.416666,NaN,NaN,NaN,2021-11-01,Monday,NaN
3,50,297749,1,MLKS,7.0,TO DOWNTOWN,5.766666,NaN,NaN,NaN,2021-11-01,Monday,NaN
4,50,297749,2,MCC5_11,5.0,TO DOWNTOWN,0.000000,NaN,NaN,NaN,2021-11-01,Monday,NaN


In [83]:
# 1. Filter dataframe to provide only entries for 'ROUTE_ABBR' 3
# 2. Filter dataframe to provide only entries for 'TRIP_EDGE' as 0 or 1
trip_edge = [0, 1]
impact_rt_3 = impact_df[(impact_df['ROUTE_ABBR'] == 3) & impact_df['TRIP_EDGE'].isin(trip_edge)]
impact_rt_3.head()

,ROUTE_ABBR,TRIP_ID,TRIP_EDGE,TIME_POINT_ABBR,ROUTE_STOP_SEQUENCE,ROUTE_DIRECTION_NAME,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,DATE,WEEKDAY,HDWY_DEV_PCT
1257,3,298945,1,WHBG,6.0,TO DOWNTOWN,9.816666,NaN,NaN,NaN,2021-11-01,Monday,NaN
1258,3,298945,0,HRWB,5.0,TO DOWNTOWN,0.000000,NaN,NaN,NaN,2021-11-01,Monday,NaN
1259,3,298945,0,WE31,4.0,TO DOWNTOWN,0.000000,NaN,NaN,NaN,2021-11-01,Monday,NaN
1260,3,298945,0,WE23,3.0,TO DOWNTOWN,0.000000,NaN,NaN,NaN,2021-11-01,Monday,NaN
1262,3,298934,1,MCC5_5,2.0,FROM DOWNTOWN,10.716666,NaN,NaN,NaN,2021-11-01,Monday,NaN


In [105]:
# 1. Use .groupby() to group table in order of: 'ROUTE_ABBR', 'TRIP_EDGE', 'WEEKDAY'
# 2. Calculate the averages for: 'DWELL_IN_MINS', 'HDWY_DEV'
rt_3_table = impact_rt_3.groupby(['ROUTE_ABBR', 'TRIP_EDGE', 'ROUTE_STOP_SEQUENCE', 'WEEKDAY']).agg({ 'DWELL_IN_MINS':['mean'], 'HDWY_DEV_PCT':['mean']})
rt_3_table

DWELL_IN_MINS HDWY_DEV_PCT
                                                            mean         mean
ROUTE_ABBR TRIP_EDGE ROUTE_STOP_SEQUENCE WEEKDAY                             
3          0         2.0                 Friday         2.705000    -0.009778
                                         Monday         5.995833     0.151535
                                         Saturday       5.106666    -0.308611
                                         Sunday        11.533333    -0.185278
                                         Thursday       5.459524    -0.003853
                                         Tuesday        2.301852     0.045895
                                         Wednesday      3.000877    -0.062456
                     3.0                 Friday         0.108049     0.024723
                                         Monday         0.140591     0.013199
                                         Saturday       0.063218     0.016260
                                         Sunday         0.128740     0.016791
                                         Thursday       0.120802     0.025092
                                         Tuesday        0.115990     0.015092
                                         Wednesday      0.112220     0.027826
                     4.0                 Friday         0.436436     0.018590
                                         Monday         0.475108     0.012223
                                         Saturday       0.217994     0.016116
                                         Sunday         0.296930     0.016775
                                         Thursday       0.409869     0.028390
                                         Tuesday        0.466997     0.013818
                                         Wednesday      0.398252     0.027494
                     5.0                 Friday         0.183623     0.017314
                                         Monday         0.234413     0.013010
                                         Saturday       0.084404     0.015509
                                         Sunday         0.166489     0.015329
                                         Thursday       0.160503     0.024564
                                         Tuesday        0.150291     0.012837
                                         Wednesday      0.136735     0.026181
                     6.0                 Friday         2.704166     0.687974
                                         Monday         2.833333     0.286140
                                         Saturday       2.733333    -0.091250
                                         Sunday         1.511111     0.179167
                                         Thursday       0.000000    -0.133706
                                         Tuesday        7.900000    -0.013218
                                         Wednesday      2.933333    -0.564815
                     7.0                 Friday         0.171966     0.015477
                                         Monday         0.142236     0.017921
                                         Saturday       0.026346     0.020943
                                         Sunday         0.074090     0.010459
                                         Thursday       0.098542     0.017839
                                         Tuesday        0.103041     0.020518
                                         Wednesday      0.100966     0.029111
                     8.0                 Friday         1.888889    -0.244444
                                         Monday         0.000000    -0.108824
                                         Thursday       0.613889     0.114121
                                         Tuesday        2.675000    -0.133843
                                         Wednesday      8.530000     0.401481
           1         2.0                 Friday        11.460609     0.011627
                                         Monday        11.709165    -0.00

In [106]:
rt_3_table2 = impact_rt_3.groupby(['ROUTE_ABBR', 'TRIP_EDGE', 'ROUTE_STOP_SEQUENCE', 'WEEKDAY']).agg({ 'ROUTE_STOP_SEQUENCE':['value_counts']})
rt_3_table2

ROUTE_STOP_SEQUENCE
                                                                              value_counts
ROUTE_ABBR TRIP_EDGE ROUTE_STOP_SEQUENCE WEEKDAY   ROUTE_STOP_SEQUENCE                    
3          0         2.0                 Friday    2.0                                  11
                                         Monday    2.0                                  16
                                         Saturday  2.0                                   5
                                         Sunday    2.0                                   1
                                         Thursday  2.0                                  14
                                         Tuesday   2.0                                  10
                                         Wednesday 2.0                                  21
                     3.0                 Friday    3.0                                6325
                                         Monday    3.0                                6363
                                         Saturday  3.0                                4364
                                         Sunday    3.0                                4094
                                         Thursday  3.0                                6354
                                         Tuesday   3.0                                6547
                                         Wednesday 3.0                                6396
                     4.0                 Friday    4.0                                6325
                                         Monday    4.0                                6361
                                         Saturday  4.0                                4364
                                         Sunday    4.0                                4094
                                         Thursday  4.0                                6355
                                         Tuesday   4.0                                6547
                                         Wednesday 4.0                                6396
                     5.0                 Friday    5.0                                6325
                                         Monday    5.0                                6362
                                         Saturday  5.0                                4364
                                         Sunday    5.0                                4094
                                         Thursday  5.0                                6355
                                         Tuesday   5.0                                6547
                                         Wednesday 5.0                                6396
                     6.0                 Friday    6.0                                   6
                                         Monday    6.0                                   9
                                         Saturday  6.0                                   2
                                         Sunday    6.0                                   3
                                         Thursday  6.0                                   3
                                         Tuesday   6.0                                   4
                                         Wednesday 6.0                                   2
                     7.0                 Friday    7.0                                2228
                                         Monday    7.0                                2225
                                         Saturday  7.0                                1524
                                         Sunday    7.0                                1488
                                         Thursday  7.0                                2232
                                         Tuesday   7.0                                2304
                                         Wednesday 7.0             

#### Q3:
How closely does lateness (ADHERENCE) correlate to headway?

#### Q4:
> What is the relationship between distance or time travelled since the start of a given trip and the headway deviation?

> Does headway become less statble the further along the route the bus has travelled?

#### Q5:
> How much of a factor does the driver have on headway and on-time performance? The driver is indicated by the 'OPERATOR' variable.

#### Q6:
How does direction of travel ('ROUTE_DIRECTION_NAME'), route ('ROUTE_ABBR'), or location ('TIME_POINT_ABBR') affect the headway and on-time performance?

#### Q7:
> How does time of day or day of week affect headway ('HWDY_DEV') and on-time performance ('ADHERENCE')?
>+ `.groupby().agg({:})` to group by route, day of week, adherence
>+ `.groupby().agg({:})` to group by route, day of week, headway deviation

> Can you detect an impact of school schedule on headway deviation (for certain routes and at certain times of day)?
>+ What are we going to define the school year as?
>+ What are we going to define rush hour as?

In [ ]:
# Convert 'DATE' to datetime datatype
# Create for loop to determine the weekday of the 'DATE' column, create 'WEEKDAY' column
headway_slim['DATE'] = pd.to_datetime(headway_slim['DATE'])
headway_slim['WEEKDAY'] = [calendar.day_name[x.weekday()] for x in headway_slim['DATE']]
headway_slim.head()

In [ ]:
# Re-arrange columns to move 'WEEKDAY' closer to 'DATE'
headway_slim = headway_slim.reindex(columns=['CALENDAR_ID', 'SERVICE_ABBR', 'ADHERENCE_ID', 'DATE', 'WEEKDAY', 'ROUTE_ABBR', 'BLOCK_ABBR', 'OPERATOR', 'TRIP_ID', 'OVERLOAD_ID', 'ROUTE_DIRECTION_NAME', 'TIME_POINT_ABBR', 'ROUTE_STOP_SEQUENCE', 'TRIP_EDGE', 'LATITUDE', 'LONGITUDE', 'SCHEDULED_TIME', 'ACTUAL_ARRIVAL_TIME', 'ACTUAL_DEPARTURE_TIME', 'ADHERENCE', 'ADHERENCE_QUARTILE', 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 'HDWY_DEV', 'HDWY_QUARTILE'])
headway_slim.head()

In [ ]:
# Create DataFrame for busses that are running while Metro Public Schools are on break, holidays
holidays = ['2021-11-11', '2021-11-24', '2021-11-25', '2021-11-26', '2021-12-20', '2021-12-21', '2021-12-22', '2021-12-23', '2021-12-24','2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31', '2022-1-3', '2022-1-4', '2022-1-5', '2022-1-6', '2022-1-17', '2022-3-14', '2022-3-15', '2022-3-16', '2022-3-17', '2022-3-18', '2022-4-15', '2022-5-3']
headway_holidays = headway_slim[headway_slim['DATE'].isin(holidays)]
headway_holidays.head()

In [ ]:
# Create DataFrame for busses that are running while Metro Public Schools are not on break, holidays
holidays = ['2021-11-11', '2021-11-24', '2021-11-25', '2021-11-26', '2021-12-20', '2021-12-21', '2021-12-22', '2021-12-23', '2021-12-24','2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31', '2022-1-3', '2022-1-4', '2022-1-5', '2022-1-6', '2022-1-17', '2022-3-14', '2022-3-15', '2022-3-16', '2022-3-17', '2022-3-18', '2022-4-15', '2022-5-3']
headway_not_holidays = headway_slim[~headway_slim['DATE'].isin(holidays)]
headway_not_holidays.head()

In [ ]:
# For loop to determine the whether time is rush hour using 'SCHEDULED_TIME' column, create 'RUSH_HOUR' column
headway_slim['RUSH_HOUR'] = ''
for ind, row in headway_slim.iterrows():
    if row['SCHEDULED_TIME'] >= '07:00:00' and row['SCHEDULED_TIME'] <= '09:30:00':
        headway_slim.loc[ind, 'RUSH_HOUR'] = 'Morning Rush'
    elif row['SCHEDULED_TIME'] >= '16:00:00' and row['SCHEDULED_TIME'] <= '18:00:00':
        headway_slim.loc[ind, 'RUSH_HOUR'] ='Evening Rush'
    else:
        headway_slim.loc[ind, 'RUSH_HOUR'] ='Not Rush'
headway_slim.head()

In [ ]:
# Create DataFrame for buses traveling during rush hour periods
headway_rush = headway_slim.loc[(headway_slim['RUSH_HOUR'] == 'Morning Rush') | (headway_slim['RUSH_HOUR'] == 'Evening Rush')]
headway_rush.head()

In [ ]:
# Create DataFrame for buses not traveling during rush hour periods
headway_not_rush = headway_slim.loc[headway_slim['RUSH_HOUR'] == 'Not Rush']
headway_not_rush.head()

#### Q8:
> Does weather have any effect on headway ('HDWY_DEV') or on-time performance ('ADHERENCE')?

> To help answer this question, the file bna_2022.csv contains historical weather data recorded at Nashville International Airport.